# Initializing the data

Change the link to manifest.csv, table 1,2 and 3 in the following cell

In [ ]:
from glob import glob

In [ ]:
manifest_csv='media/manifests/manifest.csv'
table1=glob('data/products/VLASS*Catalogue*.csv')[-1]
table3=glob('data/products/CIRADA_VLASS*table3_subtile_info_*.csv')[-1]

In [1]:
!mkdir qa_outputs

In [ ]:
import pandas as pd

table1_saved_stats='saved_counts/table1_stats.csv'
table2_saved_stats='saved_counts/VLASS2SECIR_hosts_stats.csv'

table1data=pd.read_csv(table1)

table3data=pd.read_csv(table3)

typen=table1data.Component_name.iloc[0].split(' ')[0][6:8]
epoch=table1data.Component_name.iloc[0].split(' ')[0][5]

typesav,epochsav='SE','2'

## Checking table sizes

In [ ]:
import pandas as pd

def validate_table_sizes(manifest_path, table1, table3):
    # Load the manifest file and count the lines
    with open(manifest_path, 'r') as file:
        n = len(file.readlines())

    # Load Table 1 and Table 3
#     table1 = pd.read_csv(table1_path)
#     table3 = pd.read_csv(table3_path)

    # Calculate the ratios
    ratio_table1 = len(table1) / (n - 1)
    ratio_table3 = len(table3) / (n - 1)
    fout=open('qa_outputs/sizes.txt',mode='w')
    # Check if the ratios are within the expected range
    print(f"Table 1 Ratio: {ratio_table1}")
    fout.write(f"Table 1 Ratio: {ratio_table1}\n")
    print(f"Table 3 Ratio: {ratio_table3}")
    fout.write(f"Table 3 Ratio: {ratio_table3}\n")
    if 0.95<ratio_table3 < 1.05 and 86 <= ratio_table1 <= 96:
        print("Table sizes are within the expected range.")
        fout.write("Table sizes are within the expected range.\n")
    else:
        print("Table sizes are NOT within the expected range.")
        fout.write("Table sizes are NOT within the expected range.\n")

# Usage example
validate_table_sizes(manifest_csv,table1data,table3data)

# Checking stats of the table.

In [ ]:
table1databackup=table1data.copy()
import pandas as pd

def compare_future_table_with_stats(future_table, stats_file_path):
    # Columns to be excluded from the comparison
    excluded_columns = [
        'Alpha_err_corr', 'Alpha_quality_flag', 'Alpha_err_(TT1/TT0)', 
        'Best_neuron_y', 'Best_neuron_x', 'Neuron_dist', 'P_sidelobe'
    ]

    # Specific columns to check
    columns_to_check = ['Total_flux', 'Peak_flux', 'DC_Maj']

    # Load the saved statistics and the future table
    saved_stats = pd.read_csv(stats_file_path, index_col=0)
#     future_table = pd.read_csv(future_table_path)

    # Drop the specified columns from the future table and saved stats if they exist
    future_table.drop(columns=excluded_columns, axis=1, errors='ignore', inplace=True)
    saved_stats.drop(index=excluded_columns, errors='ignore', inplace=True)

    # Calculate the descriptive statistics for the future table
    future_stats = future_table.describe()
    fout=open('qa_outputs/stats.txt',mode='w')
    # Iterate only over the columns specified in columns_to_check
    for column in columns_to_check:
        if column in future_stats.columns and column in saved_stats.columns:
            # Median comparison
            saved_median = round(saved_stats.loc['50%', column], 2)
            future_median = round(future_stats.loc['50%', column], 2)
            print(f"Column: {column}")
            fout.write(f"Column: {column}\n")
            print(f"Median of {typesav}{epochsav}: {saved_median}, Median of {typen}{epoch}: {future_median}")
            fout.write(f"Median of {typesav}{epochsav}: {saved_median}, Median of {typen}{epoch}: {future_median}\n")
            # IQR comparison as a measure of the distribution width
            saved_iqr = round(saved_stats.loc['75%', column] - saved_stats.loc['25%', column], 2)
            future_iqr = round(future_stats.loc['75%', column] - future_stats.loc['25%', column], 2)
            # print(f"Comparing IQR for column: {column}")
            print(f"IQR of {typesav}{epochsav}: {saved_iqr}, IQR of  {typen}{epoch}: {future_iqr}")
            fout.write(f"IQR of {typesav}{epochsav}: {saved_iqr}, IQR of  {typen}{epoch}: {future_iqr}\n")
            print("-" * 50)
            fout.write("-" * 50+'\n')


compare_future_table_with_stats(table1data,table1_saved_stats)

In [ ]:
stats_data_1=pd.read_csv(table1_saved_stats, index_col=0)

table1data=table1databackup.copy()
stats_current_1=table1data.describe()

merged_stats=pd.merge(stats_data_1, stats_current_1, left_index=True, right_index=True)

merged_stats=merged_stats[['RA_x','RA_y','DEC_x','DEC_y','E_RA_x','E_RA_y','E_DEC_x','E_DEC_y','Total_flux_x','Total_flux_y','E_Total_flux_x','E_Total_flux_y','Peak_flux_x','Peak_flux_y','E_Peak_flux_x','E_Peak_flux_y','Maj_x','Maj_y','E_Maj_x','E_Maj_y','Min_x','Min_y','E_Min_x','E_Min_y','PA_x','PA_y','E_PA_x','E_PA_y','Isl_Total_flux_x','Isl_Total_flux_y','E_Isl_Total_flux_x','E_Isl_Total_flux_y','Isl_rms_x','Isl_rms_y','Isl_mean_x','Isl_mean_y','Resid_Isl_rms_x','Resid_Isl_rms_y','Resid_Isl_mean_x','Resid_Isl_mean_y','RA_max_x','RA_max_y','DEC_max_x','DEC_max_y','E_RA_max_x','E_RA_max_y','E_DEC_max_x','E_DEC_max_y','Xposn_x','Xposn_y','E_Xposn_x','E_Xposn_y','Yposn_x','Yposn_y','E_Yposn_x','E_Yposn_y','Xposn_max_x','Xposn_max_y','E_Xposn_max_x','E_Xposn_max_y','Yposn_max_x','Yposn_max_y','E_Yposn_max_x','E_Yposn_max_y','Maj_img_plane_x','Maj_img_plane_y','E_Maj_img_plane_x','E_Maj_img_plane_y','Min_img_plane_x','Min_img_plane_y','E_Min_img_plane_x','E_Min_img_plane_y','PA_img_plane_x','PA_img_plane_y','E_PA_img_plane_x','E_PA_img_plane_y','DC_Maj_x','DC_Maj_y','E_DC_Maj_x','E_DC_Maj_y','DC_Min_x','DC_Min_y','E_DC_Min_x','E_DC_Min_y','DC_PA_x','DC_PA_y','E_DC_PA_x','E_DC_PA_y','DC_Maj_img_plane_x','DC_Maj_img_plane_y','E_DC_Maj_img_plane_x','E_DC_Maj_img_plane_y','DC_Min_img_plane_x','DC_Min_img_plane_y','E_DC_Min_img_plane_x','E_DC_Min_img_plane_y','DC_PA_img_plane_x','DC_PA_img_plane_y','E_DC_PA_img_plane_x','E_DC_PA_img_plane_y','QL_image_RA_x','QL_image_RA_y','QL_image_DEC_x','QL_image_DEC_y','NVSS_distance_x','NVSS_distance_y','FIRST_distance_x','FIRST_distance_y','Peak_to_ring_x','Peak_to_ring_y','Duplicate_flag_x','Duplicate_flag_y','Quality_flag_x','Quality_flag_y','NN_dist_x','NN_dist_y','BMAJ_x','BMAJ_y','BMIN_x','BMIN_y','BPA_x','BPA_y']]

# merged_stats=merged_stats[['RA_x','RA_y','DEC_x','DEC_y','E_RA_x','E_RA_y','E_DEC_x','E_DEC_y','Total_flux_x','Total_flux_y','E_Total_flux_x','E_Total_flux_y','Peak_flux_x','Peak_flux_y','E_Peak_flux_x','E_Peak_flux_y','Maj_x','Maj_y','E_Maj_x','E_Maj_y','Min_x','Min_y','E_Min_x','E_Min_y','PA_x','PA_y','E_PA_x','E_PA_y','Isl_Total_flux_x','Isl_Total_flux_y','E_Isl_Total_flux_x','E_Isl_Total_flux_y','Isl_rms_x','Isl_rms_y','Isl_mean_x','Isl_mean_y','Resid_Isl_rms_x','Resid_Isl_rms_y','Resid_Isl_mean_x','Resid_Isl_mean_y','RA_max_x','RA_max_y','DEC_max_x','DEC_max_y','E_RA_max_x','E_RA_max_y','E_DEC_max_x','E_DEC_max_y','Xposn_x','Xposn_y','E_Xposn_x','E_Xposn_y','Yposn_x','Yposn_y','E_Yposn_x','E_Yposn_y','Xposn_max_x','Xposn_max_y','E_Xposn_max_x','E_Xposn_max_y','Yposn_max_x','Yposn_max_y','E_Yposn_max_x','E_Yposn_max_y','Maj_img_plane_x','Maj_img_plane_y','E_Maj_img_plane_x','E_Maj_img_plane_y','Min_img_plane_x','Min_img_plane_y','E_Min_img_plane_x','E_Min_img_plane_y','PA_img_plane_x','PA_img_plane_y','E_PA_img_plane_x','E_PA_img_plane_y','DC_Maj_x','DC_Maj_y','E_DC_Maj_x','E_DC_Maj_y','DC_Min_x','DC_Min_y','E_DC_Min_x','E_DC_Min_y','DC_PA_x','DC_PA_y','E_DC_PA_x','E_DC_PA_y','DC_Maj_img_plane_x','DC_Maj_img_plane_y','E_DC_Maj_img_plane_x','E_DC_Maj_img_plane_y','DC_Min_img_plane_x','DC_Min_img_plane_y','E_DC_Min_img_plane_x','E_DC_Min_img_plane_y','DC_PA_img_plane_x','DC_PA_img_plane_y','E_DC_PA_img_plane_x','E_DC_PA_img_plane_y','QL_image_RA_x','QL_image_RA_y','QL_image_DEC_x','QL_image_DEC_y','NVSS_distance_x','NVSS_distance_y','FIRST_distance_x','FIRST_distance_y','Peak_to_ring_x','Peak_to_ring_y','Duplicate_flag_x','Duplicate_flag_y','Quality_flag_x','Quality_flag_y','NN_dist_x','NN_dist_y','BMAJ_x','BMAJ_y','BMIN_x','BMIN_y','BPA_x','BPA_y','Best_neuron_y_x','Best_neuron_y_y','Best_neuron_x_x','Best_neuron_x_y','Neuron_dist_x','Neuron_dist_y','P_sidelobe_x','P_sidelobe_y']]

# Creating the mapping for old to new column names
new_column_names = {col: col.replace('_x', f'_{typesav}{epochsav}').replace('_y', f'_{typen}{epoch}') for col in merged_stats.columns}

# Renaming the columns
merged_stats.rename(columns=new_column_names, inplace=True)
merged_stats=merged_stats.T

In [ ]:
pd.set_option('display.max_columns', None)
# Set to display all the rows
pd.set_option('display.max_rows', None)
# Optionally, set to display the full content in each cell
pd.set_option('display.max_colwidth', None)


## Stats comparison

In [ ]:
merged_stats

In [ ]:
merged_stats.to_csv('qa_outputs/merged_stats.csv')

# Plotting the comparison plots

In [ ]:
!pip install ipykernel pandas matplotlib seaborn numpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

def plot_histograms(data, columns, output_dir, saved_counts_dir):
    """
    Generate histograms for the specified columns in the dataset using the bin edges from previously saved histograms
    and overlay the histogram from previously saved counts and bin edges.
    """
    for col in columns:
        plt.figure(figsize=(10, 6))

        # Load the previous histogram's bin edges
        edges_file = f"{saved_counts_dir}/{col}_se2_bin_edges.csv"
        if not os.path.exists(edges_file):
            print(f"Bin edges file for {col} not found.")
            continue

        bin_edges = np.loadtxt(edges_file, delimiter=",")
        
        # Filter out non-positive values if present and ensure the data fits within the bin range
        filtered_data = data[col][(data[col] > 0) & (data[col] <= bin_edges[-1])]

        # Plotting the new histogram using the loaded bin edges
        plt.hist(filtered_data, bins=bin_edges, edgecolor='black', alpha=0.5, label=typen+epoch, density=True)
#         print(bin_edges)
        # Load and overlay the previous histogram
        counts_file = f"{saved_counts_dir}/{col}_se2_counts.csv"
        if os.path.exists(counts_file):
            counts = np.loadtxt(counts_file, delimiter=",")
            # Ensure that counts are normalized for the area to match
            plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), color='orange', edgecolor='black', align='edge', alpha=0.5, label=typesav+epochsav)

        plt.title(f'Histogram of {col}')
        plt.xlabel(col)
        plt.ylabel('Normalized Frequency')
        plt.xscale('log')  # Setting the x-axis to a logarithmic scale
        plt.yscale('log')  # Setting the y-axis to a logarithmic scale for normalized counts
        plt.legend()
        plt.savefig(f"{output_dir}/{col}_histogram.png")
        plt.show()


# Usage example - adjust the paths and directories as needed
data = table1data
os.system('rm -rf qa_outputs')
os.mkdir('qa_outputs')
output_directory = 'qa_outputs'


saved_counts_directory = 'saved_counts'

# Example columns to plot and analyze - modify as per your data
flux_columns = ['Total_flux', 'Peak_flux']
size_columns = ['DC_Maj']
# spectral_index_columns = ['Spectral_index']
quality_flag_column = 'Quality_flag'

# Generating plots with overlay
plot_histograms(data, flux_columns + size_columns, output_directory, saved_counts_directory)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_bar_chart(data, column, output_dir, saved_counts_file):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    fig, ax = plt.subplots(figsize=(10, 6))
#     current_value_counts = data[column].value_counts().sort_index()
    current_value_counts = data[column].value_counts(normalize=True).sort_index()
    
    current_value_counts.plot(kind='bar', ax=ax, color='blue', alpha=0.5, label=typen+epoch)

    saved_value_counts = pd.read_csv(saved_counts_file, sep='\t', index_col=0)#.sort_index()
    saved_value_counts['count'] /= saved_value_counts['count'].sum()  # Normalizing
    saved_value_counts = saved_value_counts.sort_index()

    saved_value_counts.plot(kind='bar', ax=ax, color='red', alpha=0.5,label=typesav+epochsav)

    plt.title(f'Bar Chart of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.yscale('log')
    plt.legend()
    plt.savefig(f"{output_dir}/{column}_barchart.png")
    plt.show()
# Usage example - Replace with your actual file paths
saved_counts_file = 'saved_counts/Quality_flag_value_counts.txt'  # Path to the saved counts
column = 'Quality_flag'  # The column to plot
output_directory = 'qa_outputs'  # Directory to save the plot
current_data = table1data

# Generate the bar chart and overplot with saved data
plot_bar_chart(current_data, column, output_directory, saved_counts_file)


In [ ]:
# Appendix

## The code which was used for saving the bin edges and counts for the histogram


# import numpy as np
# import pandas as pd
# df1=pd.read_csv('/home/biny/Dropbox/pythoncodes/Uom/uomwebsite/CIRADA_VLASS2SEv2_table1_components.csv')
# # Assuming counts and bin_edges are obtained from the np.histogram function
# filtered_data=df1[df1.Total_flux>0]['Total_flux']
# # Determine the range for logspace
# min_val = filtered_data.min()
# max_val = filtered_data.max()
# bins = 10**(np.linspace(np.log10(min_val), np.log10(max_val), 30))

# counts, bin_edges = np.histogram(filtered_data, bins=bins, density=True)

# # Save counts and bin_edges to files
# np.savetxt("saved_counts/Total_flux_se2_counts.csv", counts, delimiter=",", fmt='%1.15e')
# np.savetxt("saved_counts/Total_flux_se2_bin_edges.csv", bin_edges, delimiter=",", fmt='%1.15e')

# filtered_data=df1[df1.Peak_flux>0]['Peak_flux']
# # Determine the range for logspace
# min_val = filtered_data.min()
# max_val = filtered_data.max()
# bins = 10**(np.linspace(np.log10(min_val), np.log10(max_val), 30))

# counts, bin_edges = np.histogram(filtered_data, bins=bins, density=True)

# # Save counts and bin_edges to files
# np.savetxt("saved_counts/Peak_flux_se2_counts.csv", counts, delimiter=",", fmt='%1.15e')
# np.savetxt("saved_counts/Peak_flux_se2_bin_edges.csv", bin_edges, delimiter=",", fmt='%1.15e')

# filtered_data=df1[df1.DC_Maj>0]['DC_Maj']
# # Determine the range for logspace
# min_val = filtered_data.min()
# max_val = filtered_data.max()
# bins = 10**(np.linspace(np.log10(min_val), np.log10(max_val), 30))

# counts, bin_edges = np.histogram(filtered_data, bins=bins, density=True)


# np.savetxt("saved_counts/DC_Maj_se2_counts.csv", counts, delimiter=",", fmt='%1.15e')
# np.savetxt("saved_counts/DC_Maj_se2_bin_edges.csv", bin_edges, delimiter=",", fmt='%1.15e')
# column='Quality_flag'
# value_counts = df1[column].value_counts()
# value_counts.to_csv(f"saved_counts/{column}_value_counts.txt", sep='\t')
# value_counts

